In [2]:
from Architectures.GAN import GAN
from Data.load import load
from matplotlib import pyplot as plt
import numpy as np

In [3]:
train, test = load("Pre-processed", take=100)

Loading images: 100%|██████████| 100/100 [00:41<00:00,  2.40it/s]


In [4]:
gan_model = GAN(latent_dim=100)

C:\Users\PieHu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\PieHu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
C:\Users\PieHu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer

In [ ]:
gan_model.train(train, epochs=10000, batch_size=128)

In [ ]:
latent_space = gan_model.encode(test)
decoded_image = gan_model.decode(latent_space)

print("Original image shape:", test.shape)
print("Latent space shape:", latent_space.shape)
print("Decoded image shape:", decoded_image.shape)